# Mesa Data Preparation
So, I've finally decided that there's no way to circumvent the need to use MESA (Modules for Experimental Stellar Astrophysics) to do this research.

This means we will have to prepare the initial simulation data for MESA. We will randomly sample 1000 stars from the Gaia DR3 dataset retrieved using the following ADQL query:

```sql
SELECT TOP 1000000
	gaia.source_id, 
	gaia.ra as ra,
	gaia.dec as dec,
	parameters.mass_flame AS mass,
	parameters.radius_gspphot AS radius,
	parameters.age_flame as age
FROM gaiadr3.astrophysical_parameters AS parameters
INNER JOIN gaiadr3.gaia_source as gaia
ON gaia.source_id = parameters.source_id
WHERE 
	parameters.mass_flame IS NOT NULL
	AND parameters.fem_gspspec IS NOT NULL
	AND parameters.age_flame IS NOT NULL
	AND parameters.evolstage_flame > 100 -- 100 = Zero age main sequence star
	AND parameters.evolstage_flame < 360 -- 360 = main sequence turn off
```

## The Simulations
We will be running an evolutionary simulation for each star up until its current age (estimated by Gaia).
We will provide metallicity, initial mass, age, and initial helium (calculated from the metallicity) as inputs to the simulation.

We will export the simulation parameters to a csv `data.csv` file and use that as input for parallel execution using GNU Parallel.
The data will be exported to the `parallel` directory which will then be used to run the simulations.

In [6]:
# Import libraries
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy.io import fits
import os
import requests
import gzip
from collections import namedtuple
from typing import Union
from pathlib import Path

### Import the Gaia Data and convert to a pandas DataFrame
The above query must be run in the [Gaia Archive Advanced Query Tool (ADQL)](https://gea.esac.esa.int/archive/) and then the results must be downloaded and stored in the `data` directory.

First, we must unzip the file, then we can read it into a pandas DataFrame.

In [2]:
filename = 'data/gaia_astrophysical_parameters.vot.gz'

# If unzipped file does not exist, unzip and decode it
if not os.path.exists('data/gaia_astrophysical_parameters.vot'):
    with gzip.open(filename, 'rb') as f:
        with open('data/gaia_astrophysical_params.vot', 'wb') as g:
            g.write(f.read())

In [4]:
df: pd.DataFrame = Table.read('data/gaia_astrophysical_params.vot').to_pandas()
df

,SOURCE_ID,ra,dec,log_surface_gravity,dist,mass,radius,log_fe_h_abundance,log_n_fe_abundance,log_s_fe_abundance,log_metalicity,age,evolution_stage,spectral_type
0,6030024480434041088,256.473786,-28.289514,3.9771,177.857895,1.409851,2.0905,-0.09,0.47,0.11,0.04,2.920355,316,F
1,6030111685419272576,255.038856,-28.811967,4.2980,182.385498,1.171434,1.1288,0.18,NaN,NaN,-0.54,1.242147,160,G
2,5253752950266984704,157.348507,-62.332491,3.7712,279.329987,1.670828,2.9348,-0.19,NaN,0.19,-0.00,1.923422,337,F
3,6030062409175986176,254.932569,-28.931663,3.9315,405.607208,1.477585,2.2852,-0.13,NaN,NaN,0.20,2.654922,327,F
4,5961557062324546304,265.895060,-37.946593,3.9318,342.907501,1.518954,2.1685,0.03,NaN,NaN,0.06,2.108766,277,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205584,1796082381758247552,328.159849,25.054985,4.0129,112.224403,1.343488,1.8720,-0.07,0.02,-0.04,0.25,3.229928,312,F
205585,1794976651018650624,328.512913,22.939668,4.1252,213.870605,1.105674,1.3825,-0.02,NaN,NaN,-0.05,5.438365,337,F
205586,1796123682164850688,326.907823,24.488709,3.9158,355.876312,1.487074,2.3270,-0.16,NaN,0.16,-0.11,2.510785,303,F
205587,1795069284873344640,328.001389,23.488187,4.1705,167.741104,1.119260,1.2895,0.12,NaN,NaN,-0.22,4.767622,320,F


## Spectroscopy Data (APOGEE DR17)
We can improve our MESA simulations by using more fine-grained spectroscopy data.

Download the FITS file from SDSS's (Sloan Digital Sky Survey) [cloud storage](https://data.sdss.org/sas/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStar-dr17-synspec_rev1.fits)

This may take a while to download.

In [4]:
url = 'https://data.sdss.org/sas/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStar-dr17-synspec_rev1.fits'

if not os.path.exists('data/APOGEE_DR17.fits'):
    response = requests.get(url)
response.raise_for_status()

# Store to 'data/galah_all_stars_spectroscopy.fits'
with open('data/APOGEE_DR17.fits', 'wb') as f:
    f.write(response.content)

In [5]:
# Load the data
cols = ['GAIAEDR3_SOURCE_ID', 'C_FE', 'C_FE_FLAG', 'N_FE', 'N_FE_FLAG', 'O_FE', 'O_FE_FLAG', 'NA_FE', 'NA_FE_FLAG', 'MG_FE', 'MG_FE_FLAG', 'AL_FE', 'AL_FE_FLAG', 'SI_FE', 'SI_FE_FLAG', 'S_FE', 'S_FE_FLAG', 'K_FE', 'K_FE_FLAG', 'CA_FE', 'CA_FE_FLAG', 'CR_FE', 'CR_FE_FLAG', 'MN_FE', 'MN_FE_FLAG', 'FE_H', 'FE_H_FLAG', 'NI_FE', 'NI_FE_FLAG']
tbl: pd.DataFrame = Table.read('data/APOGEE_DR17.fits', format='fits', hdu=1)
apogee_spectroscopy = tbl[cols].to_pandas()

# Neon is not measured since its spectra are in the UV so we will use the average of the [O/Fe] and [Mg/Fe] spectra since they are created in the same fusion processes as Neon
# Suggested by this paper: https://iopscience.iop.org/article/10.3847/1538-3881/ac9bfa#ajac9bfas3
apogee_spectroscopy['NE_FE'] = (apogee_spectroscopy['O_FE'] + apogee_spectroscopy['MG_FE']) / 2
apogee_spectroscopy['NE_FE_FLAG'] = apogee_spectroscopy['O_FE_FLAG'] + apogee_spectroscopy['MG_FE_FLAG'] # Arbitrary flag that will be zero if the both oxygen and magnesium are good data

apogee_spectroscopy

,GAIAEDR3_SOURCE_ID,C_FE,C_FE_FLAG,N_FE,N_FE_FLAG,O_FE,O_FE_FLAG,NA_FE,NA_FE_FLAG,MG_FE,...,CR_FE,CR_FE_FLAG,MN_FE,MN_FE_FLAG,FE_H,FE_H_FLAG,NI_FE,NI_FE_FLAG,NE_FE,NE_FE_FLAG
0,0,0.004847,0,0.124265,0,0.114938,0,0.146668,0,0.035147,...,-0.025774,0,0.040870,0,0.003463,0,0.051278,0,0.075043,0
1,538028216707715712,0.009295,0,0.151220,0,0.083402,0,0.050112,0,0.030429,...,NaN,64,NaN,64,-0.160680,0,0.007683,0,0.056915,0
2,2413929812587459072,0.061738,0,-0.111900,256,0.235343,0,-0.190826,0,0.165238,...,-0.130982,0,-0.078027,0,-0.275530,0,0.013930,0,0.200291,0
3,422596679964513792,0.112730,0,0.114560,32,0.045069,0,-0.179346,0,0.038494,...,-1.112472,0,0.063633,0,-0.252970,0,-0.108750,0,0.041782,0
4,422596679964513792,0.032651,0,-0.489060,288,0.140573,0,-1.166266,0,-0.001095,...,-0.741282,0,-0.153027,0,-0.214170,0,-0.069440,0,0.069739,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733896,2341765776376373376,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
733897,1998097371124974720,-0.060314,0,0.201880,0,0.088913,0,-0.127408,0,0.083532,...,0.038944,0,-0.000038,0,-0.236560,0,0.022203,0,0.086223,0
733898,1994741318040223232,-0.011308,0,0.209650,0,0.050574,0,0.042082,0,0.064306,...,0.024764,0,0.036437,0,0.114820,0,0.032992,0,0.057440,0
733899,6379914575198998272,-0.616490,0,-0.004530,288,-0.064257,0,0.043602,0,-0.211707,...,0.043914,0,-0.584298,0,-1.050500,0,-0.103507,0,-0.137982,0


In [6]:
## Cross-match the Gaia data with the Galah data (using the `SOURCE_ID` in the Gaia data and the `dr3_source_id` in the Galah data)
apogee_spectroscopy.rename(columns={'GAIAEDR3_SOURCE_ID': 'SOURCE_ID'}, inplace=True)
matched_df = pd.merge(df, apogee_spectroscopy[
    ['SOURCE_ID', 'C_FE', 'C_FE_FLAG', 'O_FE', 'O_FE_FLAG', 'NE_FE_FLAG', 'NE_FE_FLAG', 'MG_FE', 'MG_FE_FLAG', 'AL_FE', 'NE_FE', 'NE_FE_FLAG',
     'AL_FE_FLAG', 'SI_FE', 'SI_FE_FLAG', 'K_FE', 'K_FE_FLAG', 'CA_FE', 'CA_FE_FLAG', 'CR_FE',
     'CR_FE_FLAG', 'MN_FE', 'MN_FE_FLAG', 'NI_FE', 'NI_FE_FLAG', 'FE_H', 'FE_H_FLAG', 'N_FE','N_FE_FLAG', 'S_FE', 'S_FE_FLAG']], on='SOURCE_ID', how='inner')
matched_df.rename(columns={'C_FE': 'C', 'O_FE': 'O', 'NE_FE': 'Ne',
                           'MG_FE': 'Mg', 'AL_FE': 'Al', 'SI_FE': 'Si',
                           'K_FE': 'K', 'CA_FE': 'Ca', 
                           'CR_FE': 'Cr', 'MN_FE': 'Mn', 'NI_FE': 'Ni',
                           'FE_H': 'Fe', 'Ne_Fe': 'Ne',
                           'N_FE_FLAG': 'flag_N',
                           'FE_H_FLAG': 'flag_Fe',
                           'N_FE': 'N', 'S_FE': 'S', 'C_FE_FLAG': 'flag_C',
                           'O_FE_FLAG': 'flag_O', 'NE_FE_FLAG': 'flag_Ne',
                           'MG_FE_FLAG': 'flag_Mg', 'AL_FE_FLAG': 'flag_Al',
                           'SI_FE_FLAG': 'flag_Si', 'K_FE_FLAG': 'flag_K',
                           'CA_FE_FLAG': 'flag_Ca', 'S_FE_FLAG': 'flag_S',
                           'CR_FE_FLAG': 'flag_Cr', 'MN_FE_FLAG': 'flag_Mn',
                           'NI_FE_FLAG': 'flag_Ni'},
                  inplace=True)

matched_df = matched_df.dropna(subset=['Fe', 'N', 'O', 'C', 'Mg', 'Si', 'Ne', 'S'])
matched_df = matched_df[matched_df['flag_Fe'] == 0]
matched_df = matched_df[matched_df['flag_N'] == 0]
matched_df = matched_df[matched_df['flag_O'] == 0]
matched_df = matched_df[matched_df['flag_C'] == 0]
matched_df = matched_df[matched_df['flag_Mg'] == 0]
matched_df = matched_df[matched_df['flag_Si'] == 0]
matched_df = matched_df[matched_df['flag_S'] == 0]

matched_df

,SOURCE_ID,ra,dec,log_surface_gravity,dist,mass,radius,log_fe_h_abundance,log_n_fe_abundance,log_s_fe_abundance,...,Mn,flag_Mn,Ni,flag_Ni,Fe,flag_Fe,N,flag_N,S,flag_S
4,2268469823008193152,282.016475,75.423453,4.5217,145.762207,0.823140,0.8730,-0.00,NaN,NaN,...,0.141202,0,0.094350,0,0.211670,0,0.250330,0,-0.087508,0
8,2264752202396093440,292.152517,72.519192,4.5361,146.394302,0.815044,0.8196,0.01,NaN,NaN,...,0.040761,0,0.014277,0,0.051663,0,0.033958,0,-0.023827,0
18,2268625472622936832,280.909000,76.005640,4.4328,180.245499,0.906775,0.9562,0.00,NaN,NaN,...,0.079832,0,0.038490,0,0.116710,0,0.161760,0,-0.066233,0
19,2266607387750059648,276.122223,72.223313,4.5287,117.208000,0.857668,0.8635,0.05,NaN,NaN,...,0.055690,0,0.053450,0,0.151250,0,0.094051,0,-0.150428,0
42,2265008732201710848,292.041921,74.263128,4.4988,166.162903,0.919691,0.9020,-0.07,NaN,NaN,...,0.042765,0,0.019040,0,0.134380,0,0.033745,0,0.014305,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17014,6051325319082291072,249.930905,-22.299954,4.3875,120.769699,0.894263,1.0553,-0.07,NaN,-0.21,...,0.007943,0,0.041774,0,0.028000,0,0.026942,0,-0.074890,0
17018,6052210387286181632,245.602881,-21.537031,4.5364,123.393303,0.798299,0.8032,-0.00,NaN,NaN,...,0.022232,0,0.022975,0,0.039333,0,0.019557,0,0.054559,0
17038,1175680781122200192,216.943306,8.551573,4.4730,110.827499,0.986199,0.9424,-0.07,0.51,0.10,...,0.019909,0,0.018225,0,0.095665,0,0.117899,0,-0.070877,0
17051,1176284378646107008,217.274190,9.046392,4.3453,56.396900,0.956969,1.0322,-0.11,NaN,-0.15,...,0.044954,0,0.032282,0,0.070278,0,0.180159,0,-0.104771,0


### Necessary Constants

In [26]:
# According to this paper: https://www.aanda.org/articles/aa/full_html/2020/06/aa37694-20/aa37694-20.html
SOLAR_METALLICITY = np.log10(0.017)

# Gases produced by the big bang were approximately 24.9% helium, still according to the above paper
Y_PROTO = 0.249
Y_SUN = 0.279
Z_SUN = 0.0122

NUM_STARS = 3000 
SUN_HE_ABUNDANCE = 10.93

HYDROGEN_MASS = 1.00784
HELIUM_MASS = 4.002602
CARBON_MASS = 12.011
NITROGEN_MASS = 14.0067
OXYGEN_MASS = 15.999
NEON_MASS = 20.1797
SODIUM_MASS = 22.989769
MAGNESIUM_MASS = 24.305
ALUMINUM_MASS = 26.981539
SILICON_MASS = 28.0855
SUlFUR_MASS = 32.065
POTASSIUM_MASS = 39.0983
CALCIUM_MASS = 40.078
CHROMIUM_MASS = 51.9961
MANGANESE_MASS = 54.938044
IRON_MASS = 55.845
NICKEL_MASS = 58.6934

masses = {
    'H': HYDROGEN_MASS,
    'He': HELIUM_MASS,
    'C': CARBON_MASS,
    'N': NITROGEN_MASS,
    'O': OXYGEN_MASS,
    'Ne': NEON_MASS,
    'Na': SODIUM_MASS,
    'Mg': MAGNESIUM_MASS,
    'Al': ALUMINUM_MASS,
    'Si': SILICON_MASS,
    'S': SUlFUR_MASS,
    'K': POTASSIUM_MASS,
    'Ca': CALCIUM_MASS,
    'Cr': CHROMIUM_MASS,
    'Mn': MANGANESE_MASS,
    'Fe': IRON_MASS,
    'Ni': NICKEL_MASS,
}

SPECTROSCOPY_ATOMS = ['C', 'O', 'N', 'Ne', 'Mg', 'Al', 'Si', 'S', 'K', 'Ca', 'Cr', 'Mn', 'Fe', 'Ni']

### Stellar Data Model
The stellar data model is a object that contains the following properties for a star:
- mass (mSun)
- relative abundances
- mass fractions (calculated from relative abundances)
- metallicity (calculated from mass fractions)
- age
- radius (rSun)

A stellar data model is the input for a MESA simulation and contains all necessary information to run a stellar evolution simulation.

In [8]:
Abundance = namedtuple('Abundance', ['atom', 'relAbundance'])  # Atom name, relative abundance
AbundanceVal = namedtuple('AbundanceVal', ['aMass', 'abundance'])

### Solar Baseline Model
The solar data model is used to calculate metallicities and absolute abundances for any given star.

The solar model is based on the abundances of the Sun as calculated in this [paper](https://iopscience.iop.org/article/10.3847/1538-3881/ac9bfa#ajac9bfas3).

In [9]:
class SunModel:
    def __init__(self):
        self.mass = 1
        self.abundances = {
            'H': AbundanceVal(HYDROGEN_MASS, 12),
            'He': AbundanceVal(HELIUM_MASS, SUN_HE_ABUNDANCE),
            'C': AbundanceVal(CARBON_MASS, 8.39),
            'N': AbundanceVal(NITROGEN_MASS, 7.78),
            'O': AbundanceVal(OXYGEN_MASS, 8.66),
            'Ne': AbundanceVal(NEON_MASS, 7.84),
            'Na': AbundanceVal(SODIUM_MASS, 6.17),
            'Mg': AbundanceVal(MAGNESIUM_MASS, 7.53),
            'Al': AbundanceVal(ALUMINUM_MASS, 6.37),
            'Si': AbundanceVal(SILICON_MASS, 7.51),
            'S': AbundanceVal(SUlFUR_MASS, 7.14),
            'K': AbundanceVal(POTASSIUM_MASS, 5.08),
            'Ca': AbundanceVal(CALCIUM_MASS, 6.31),
            'Cr': AbundanceVal(CHROMIUM_MASS, 5.64),
            'Mn': AbundanceVal(MANGANESE_MASS, 5.39),
            'Fe': AbundanceVal(IRON_MASS, 7.45),
            'Ni': AbundanceVal(NICKEL_MASS, 6.23),
        }
        self.age = 4.57e9
        self.radius = 1.0
        self.mass_fracs = self.get_mass_fracs()

    def get_mass_fracs(self):
        return {
            'y': Y_SUN,
            'z': Z_SUN
        }

    @property
    def z_frac(self):
        return self.mass_fracs['z']

    @property
    def y_frac(self):
        return self.mass_fracs['y']

    @property
    def abundance_np(self):
        _pyarr = []
        for abundance in self.abundances.values():
            _pyarr.append((abundance.abundance, abundance.aMass))

        return np.array(_pyarr)

### Stellar Data Model
The stellar data model represents a star's simulation data.

In [17]:
# Solar model and calculation constants
sun_model = SunModel()
sun_abundances = sun_model.abundance_np
sun_metals = sun_abundances[2:]
sun_metal = np.sum(10 ** sun_metals[:, 0] * sun_metals[:, 1])

array([12.011   , 14.0067  , 15.999   , 20.1797  , 22.989769, 24.305   ,
       26.981539, 28.0855  , 32.065   , 39.0983  , 40.078   , 51.9961  ,
       54.938044, 55.845   , 58.6934  ])

In [34]:
class StellarDataModel:
    def __init__(self, mass: float, relative_abundances: list[Abundance], age: float, radius: float, rel_iron: float,
                 iters: Union[int, 'converge'] = 'converge', convergence_thresh: float = 1e-7):
        """
        :param mass: Mass of star in MSun 
        :param relative_abundances: List of relative abundances for each element
        :param age: Age of star in Gya
        :param rel_iron: Relative abundance of iron
        :param radius: Radius of star in RSun
        :param iters: Iteration number or 'converge' to use the convergence criteria
        :param convergence_thresh: Convergence threshold (difference between iterations must be less than this)
        """
        self.mass = mass
        self.abundances = StellarDataModel.get_abundances(relative_abundances, rel_iron)
        self.age = age
        self.mass_fracs = self.get_mass_fracs(iters, convergence_thresh)
        self.radius = radius

    @property
    def y_frac(self):
        return self.mass_fracs['y']

    @property
    def z_frac(self):
        return self.mass_fracs['z']

    # Iteratively calculate the mass fractions for each element
    def get_mass_fracs(self, iters: Union[int, 'converge'] = 'converge', convergence_thresh: float = 1e-7):
        idx_to_atom = {}
        _pyarr = []
        for (atom, abundance) in self.abundances.items():
            _pyarr.append((abundance.abundance, abundance.aMass))
            idx_to_atom[len(_pyarr) - 1] = atom

        arr = np.array(_pyarr)

        metal_star = np.sum(10 ** arr[:, 0] * arr[:, 1])
        delta = metal_star / sun_metal
        abundance_helium = np.log10(10 ** SUN_HE_ABUNDANCE * delta)

        stack_args = [
            np.array([
                (12, HYDROGEN_MASS),
                (abundance_helium, HELIUM_MASS),
            ]),
            arr,
        ]
        
        star_new = np.vstack(stack_args)

        f_H_star = np.log10(np.sum(10 ** star_new[:, 0] * star_new[:, 1]))

        # Ratio of number of nucleons of element to all nucleons, Y_Q
        atom_fracs = 10 ** (arr[:, 0] - f_H_star)
        
        # Mass fractions, X_Q
        mass_fracs = atom_fracs * arr[:, 1]
        
        # Z value
        metallicity = np.sum(mass_fracs)

        def calc_z_iter(arr: np.ndarray, z: float) -> float:
            delta = z / sun_model.z_frac
            abundance_helium = np.log10(10 ** SUN_HE_ABUNDANCE * delta)
            
            stack_args = [
                np.array([
                    (12, HYDROGEN_MASS),
                    (abundance_helium, HELIUM_MASS),
                ]),
                arr,
            ]
            
            star_new = np.vstack(stack_args)

            f_H_star = np.log10(np.sum(10 ** star_new[:, 0] * star_new[:, 1]))
            
            atom_fracs = 10 ** (arr[:, 0] - f_H_star)
            mass_fracs = atom_fracs * arr[:, 1]
            metallicity = np.sum(mass_fracs)
            return metallicity, mass_fracs

        z_last = float('inf')
        z_curr = metallicity

        if iters == 'converge':
            iters = 0

            while np.abs(z_curr - z_last) > convergence_thresh:
                z_last = z_curr
                z_curr, mass_fracs = calc_z_iter(arr, z_last)
                iters += 1

            y_frac = Y_PROTO + 1.33 * z_curr

            mass_fractions = {}
            for i in range(len(mass_fracs)):
                mass_fractions[idx_to_atom[i]] = mass_fracs[i]
                
            mass_fractions['y'] = y_frac
            mass_fractions['z'] = z_curr

            return mass_fractions
        else:
            for i in range(iters - 1):
                z_last = z_curr
                z_curr, mass_fracs = calc_z_iter(arr, z_last)

            y_frac = Y_PROTO + 1.33 * z_curr

            mass_fractions = {}
            for i in range(len(mass_fracs)):
                mass_fractions[idx_to_atom[i]] = mass_fracs[i]

            mass_fractions['y'] = y_frac
            mass_fractions['z'] = z_curr

            return mass_fractions

    @staticmethod
    def get_abundances(relative_abundances: list[Abundance], iron_abundance) -> dict[str, AbundanceVal]:
        abundances = {}
        for rel_ab in relative_abundances:
            # Q/Fe = Q/H - Fe/H; Thus Q/H = Fe/H + Q/Fe
            if rel_ab.atom == 'Fe':
                converted_as_base_hydrogen = iron_abundance
            else:
                converted_as_base_hydrogen = iron_abundance + rel_ab.relAbundance
            
            abundances[rel_ab.atom] = AbundanceVal(masses[rel_ab.atom],
                                                   converted_as_base_hydrogen + sun_model.abundances[rel_ab.atom].abundance)

        return abundances

    def to_mesa_data(self):
        data = {
            'mass': self.mass,
            'y': self.y_frac,
            'z': self.z_frac,
            'age': self.age * 1e9,
            'radius': self.radius,
        }

        for (atom, mass_frac) in self.mass_fracs.items():
            data[f'{atom}_mass_frac'] = mass_frac

        return data

In [41]:
model = StellarDataModel(1, [
    Abundance('C', 0),
    Abundance('N', 0),
    Abundance('O', 0),
    Abundance('Ne', 0),
    Abundance('Mg', 0),
    Abundance('Al', 0),
    Abundance('Si', 0),
    Abundance('S', 0),
    Abundance('K', 0),
    Abundance('Ca', 0),
    Abundance('Cr', 0),
    Abundance('Mn', 0),
    Abundance('Fe', 0),
    Abundance('Ni', 0),
], 4.603, 1, 0)
model.to_mesa_data()

12.134926914497495
[1.79917366e-04 4.41644764e-05 3.35021814e-04 5.07076035e-05
 2.48355102e-05 1.71819751e-06 2.37177281e-05 1.01174970e-05
 8.81197153e-08 1.49648747e-06 3.19943348e-07 1.79917366e-07
 2.06572776e-05 1.24472406e-06]


{'mass': 1,
 'y': 0.26614551156103283,
 'z': 0.01289136207596453,
 'age': 4603000000.0,
 'radius': 1,
 'C_mass_frac': 0.002298521337992226,
 'N_mass_frac': 0.0006579686474329167,
 'O_mass_frac': 0.005701146634094602,
 'Ne_mass_frac': 0.0010883888359337476,
 'Mg_mass_frac': 0.0006420443052462082,
 'Al_mass_frac': 4.931012354120476e-05,
 'Si_mass_frac': 0.0007085190518433602,
 'S_mass_frac': 0.00034506476651154685,
 'K_mass_frac': 3.664605655157166e-06,
 'Ca_mass_frac': 6.379335070391013e-05,
 'Cr_mass_frac': 1.769457529520846e-05,
 'Mn_mass_frac': 1.0513384930609901e-05,
 'Fe_mass_frac': 0.001227025725478864,
 'Ni_mass_frac': 7.770673130496784e-05,
 'y_mass_frac': 0.26614551156103283,
 'z_mass_frac': 0.01289136207596453}

In [13]:
# Load past data to exclude from the available stars (don't want to simulate stars that have already been simulated)
glob = Path('simulate/past_data/').glob('*.csv')
dfs = []
for file in glob:
    dfs.append(pd.read_csv(file, index_col=0))

if len(dfs) > 0:
    past_data = pd.concat(dfs)
else:
    past_data = pd.DataFrame()

# Remove the stars that have already been simulated
available_stars = matched_df[~matched_df.SOURCE_ID.isin(past_data.index)]
available_stars = available_stars.drop_duplicates(subset=['SOURCE_ID'])
available_stars

,SOURCE_ID,ra,dec,log_surface_gravity,dist,mass,radius,log_fe_h_abundance,log_n_fe_abundance,log_s_fe_abundance,...,Mn,flag_Mn,Ni,flag_Ni,Fe,flag_Fe,N,flag_N,S,flag_S
4,2268469823008193152,282.016475,75.423453,4.5217,145.762207,0.823140,0.8730,-0.00,NaN,NaN,...,0.141202,0,0.094350,0,0.211670,0,0.250330,0,-0.087508,0
8,2264752202396093440,292.152517,72.519192,4.5361,146.394302,0.815044,0.8196,0.01,NaN,NaN,...,0.040761,0,0.014277,0,0.051663,0,0.033958,0,-0.023827,0
18,2268625472622936832,280.909000,76.005640,4.4328,180.245499,0.906775,0.9562,0.00,NaN,NaN,...,0.079832,0,0.038490,0,0.116710,0,0.161760,0,-0.066233,0
19,2266607387750059648,276.122223,72.223313,4.5287,117.208000,0.857668,0.8635,0.05,NaN,NaN,...,0.055690,0,0.053450,0,0.151250,0,0.094051,0,-0.150428,0
42,2265008732201710848,292.041921,74.263128,4.4988,166.162903,0.919691,0.9020,-0.07,NaN,NaN,...,0.042765,0,0.019040,0,0.134380,0,0.033745,0,0.014305,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17014,6051325319082291072,249.930905,-22.299954,4.3875,120.769699,0.894263,1.0553,-0.07,NaN,-0.21,...,0.007943,0,0.041774,0,0.028000,0,0.026942,0,-0.074890,0
17018,6052210387286181632,245.602881,-21.537031,4.5364,123.393303,0.798299,0.8032,-0.00,NaN,NaN,...,0.022232,0,0.022975,0,0.039333,0,0.019557,0,0.054559,0
17038,1175680781122200192,216.943306,8.551573,4.4730,110.827499,0.986199,0.9424,-0.07,0.51,0.10,...,0.019909,0,0.018225,0,0.095665,0,0.117899,0,-0.070877,0
17051,1176284378646107008,217.274190,9.046392,4.3453,56.396900,0.956969,1.0322,-0.11,NaN,-0.15,...,0.044954,0,0.032282,0,0.070278,0,0.180159,0,-0.104771,0


### Prepare Dataset
Now we will prepare the dataset for MESA.

First, we will need to convert the age to a time in years. (Convert from Gya to years)
Next, we will need to calculate the initial helium abundance from the metallicity. (y)
Finally, we will add the mass and metallicity to the dataframe. (mass and z respectively)

We will also include the source_id as a column in the dataframe.

In [75]:
def format_test(test_bool):
    return '✔' if test_bool else '✘'


cols = ['source_id', 'mass', 'age', 'y', 'z', 'radius']
for atom in SPECTROSCOPY_ATOMS:
    cols.append(f'{atom}_mass_frac')
dataset = pd.DataFrame(columns=cols)
dataset.set_index('source_id', inplace=True)
df_len = len(available_stars)

# Inference test conditions
print(
    f'The sample is independent if the sample size is less than {df_len / 10}. {NUM_STARS} is less than {df_len / 10}? {format_test(NUM_STARS < df_len)}')
print(
    f'The sample is normal if the sample size is greater than 30 or the sampling or population distribution is normal. We do not know the density profiles of the stars so we will have to rely on the Central Limit Theorem. {NUM_STARS} is greater than 30? {format_test(NUM_STARS > 30)}')
print(f'The sample is random because we used NumPy random to select indices. ✔')

print()
print(f'The steps for inference have been met? {format_test(NUM_STARS < df_len / 10 and NUM_STARS > 30)}')

if NUM_STARS < df_len:
    random_indices = np.random.choice(df_len, NUM_STARS, replace=False)
else:
    random_indices = np.arange(df_len)

for i in random_indices:
    df_row = available_stars.iloc[i]

    abundances: list[Abundance] = []
    # Create stellar data model
    
    for atom in SPECTROSCOPY_ATOMS:
        val = df_row[f'{atom}']
        flag = df_row[f'flag_{atom}']
        if atom == 'Ne':
            flag = df_row['flag_Ne'].iloc[0]
        if (not np.isnan(val) 
                and flag == 0):
            abundances.append(Abundance(atom, val))
    
    # Create stellar data model
    stellar_data_model = StellarDataModel(df_row['mass'], abundances, df_row['age'], df_row['radius'], df_row['Fe'])
    dataset.loc[df_row.SOURCE_ID] = stellar_data_model.to_mesa_data()
    
    if dataset.loc[df_row.SOURCE_ID] is None:
        print(f'Failed to create stellar data model for {df_row.SOURCE_ID}')
    
dataset

The sample is independent if the sample size is less than 189.5. 3000 is less than 189.5? ✘
The sample is normal if the sample size is greater than 30 or the sampling or population distribution is normal. We do not know the density profiles of the stars so we will have to rely on the Central Limit Theorem. 3000 is greater than 30? ✔
The sample is random because we used NumPy random to select indices. ✔

The steps for inference have been met? ✘


,mass,age,y,z,radius,C_mass_frac,O_mass_frac,N_mass_frac,Ne_mass_frac,Mg_mass_frac,Al_mass_frac,Si_mass_frac,S_mass_frac,K_mass_frac,Ca_mass_frac,Ti_mass_frac,Cr_mass_frac,Mn_mass_frac,Fe_mass_frac,Ni_mass_frac
source_id,,,,,,,,,,,,,,,,,,,,
2268469823008193152,0.823140,1.299812e+10,0.299962,0.026982,0.8730,0.006018,0.011929,0.002456,0.000858,0.001175,0.000133,0.001648,0.000470,0.000007,0.000090,0.000006,0.000016,0.000023,0.001997,0.000157
2264752202396093440,0.815044,1.114927e+10,0.281967,0.018413,0.8196,0.003250,0.009049,0.001145,0.000651,0.000892,0.000102,0.001152,0.000418,0.000005,0.000079,0.000005,0.000019,0.000014,0.001533,0.000100
2268625472622936832,0.906775,9.327484e+09,0.292177,0.023275,0.9562,0.004567,0.011466,0.001681,0.000768,0.000981,0.000108,0.001372,0.000414,0.000006,0.000076,0.000004,0.000020,0.000016,0.001677,0.000116
2266607387750059648,0.857668,9.406262e+09,0.293008,0.023670,0.8635,0.004733,0.011461,0.001550,0.000812,0.001095,0.000112,0.001462,0.000368,0.000006,0.000085,0.000005,0.000027,0.000017,0.001807,0.000129
2265008732201710848,0.919691,6.044249e+09,0.290172,0.022320,0.9020,0.003877,0.011381,0.001319,0.000762,0.000972,0.000099,0.001358,0.000526,0.000005,0.000086,0.000004,0.000030,0.000016,0.001767,0.000117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6051325319082291072,0.894263,1.307046e+10,0.287016,0.020817,1.0553,0.003632,0.011372,0.001036,0.000691,0.000800,0.000095,0.001243,0.000341,0.000003,0.000065,0.000004,0.000017,0.000012,0.001409,0.000098
6052210387286181632,0.798299,1.202229e+10,0.283759,0.019266,0.8032,0.004038,0.009158,0.001065,0.000655,0.000892,0.000088,0.001210,0.000481,0.000004,0.000070,0.000004,0.000015,0.000013,0.001474,0.000098
1175680781122200192,0.986199,3.324886e+09,0.285978,0.020323,0.9424,0.003306,0.010345,0.001501,0.000682,0.000855,0.000094,0.001240,0.000405,0.000005,0.000082,0.000004,0.000022,0.000014,0.001657,0.000109


In [77]:
# In case you want to scout out the original rows
available_stars.iloc[random_indices]

,SOURCE_ID,ra,dec,log_surface_gravity,dist,mass,radius,log_fe_h_abundance,log_n_fe_abundance,log_s_fe_abundance,...,Mn,flag_Mn,Ni,flag_Ni,Fe,flag_Fe,N,flag_N,S,flag_S
4,2268469823008193152,282.016475,75.423453,4.5217,145.762207,0.823140,0.8730,-0.00,NaN,NaN,...,0.141202,0,0.094350,0,0.211670,0,0.250330,0,-0.087508,0
8,2264752202396093440,292.152517,72.519192,4.5361,146.394302,0.815044,0.8196,0.01,NaN,NaN,...,0.040761,0,0.014277,0,0.051663,0,0.033958,0,-0.023827,0
18,2268625472622936832,280.909000,76.005640,4.4328,180.245499,0.906775,0.9562,0.00,NaN,NaN,...,0.079832,0,0.038490,0,0.116710,0,0.161760,0,-0.066233,0
19,2266607387750059648,276.122223,72.223313,4.5287,117.208000,0.857668,0.8635,0.05,NaN,NaN,...,0.055690,0,0.053450,0,0.151250,0,0.094051,0,-0.150428,0
42,2265008732201710848,292.041921,74.263128,4.4988,166.162903,0.919691,0.9020,-0.07,NaN,NaN,...,0.042765,0,0.019040,0,0.134380,0,0.033745,0,0.014305,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17014,6051325319082291072,249.930905,-22.299954,4.3875,120.769699,0.894263,1.0553,-0.07,NaN,-0.21,...,0.007943,0,0.041774,0,0.028000,0,0.026942,0,-0.074890,0
17018,6052210387286181632,245.602881,-21.537031,4.5364,123.393303,0.798299,0.8032,-0.00,NaN,NaN,...,0.022232,0,0.022975,0,0.039333,0,0.019557,0,0.054559,0
17038,1175680781122200192,216.943306,8.551573,4.4730,110.827499,0.986199,0.9424,-0.07,0.51,0.10,...,0.019909,0,0.018225,0,0.095665,0,0.117899,0,-0.070877,0
17051,1176284378646107008,217.274190,9.046392,4.3453,56.396900,0.956969,1.0322,-0.11,NaN,-0.15,...,0.044954,0,0.032282,0,0.070278,0,0.180159,0,-0.104771,0


In [79]:
# Export the data to a csv file
dataset.to_csv('simulate/data.csv')

### Setting up for simulations
Please be aware that you will need to download and install both GNU Parallel and MESA.

You can install GNU Parallel using `sudo dnf install parallel` or `sudo apt install parallel`, depending on your distribution.

Please follow the instructions on [MESA's website](https://docs.mesastar.org/en/latest/installation.html).

I used SDK Version **23.7.3** and MESA Version **24.03.1**.

I would highly recommend downloading the source directly from the [MESA GitHub](https://github.com/MESAHub/mesa/releases/tag/r24.03.1) 
rather than downloading it from Zenodo since it can be **slow** to download from Zenodo.

I used a **Fedora 40** distribution tailored for astronomy. I downloaded the ISO image from [here](https://fedoraproject.org/labs/astronomy/).